In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, sleep):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' +keyword+ '&locT=&locId=&jobType='
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 15, False, '/Users/jessietoib/Desktop/Awari/ds_salary_proj/chromedriver', 15)
df

Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,"Data Scientist II, Product Analytics",$145K-$163K (Glassdoor est.),"The Opportunity\n\n\nAt Livongo, data is at th...",4.7,Livongo Health\n4.7,"Mountain View, CA",-1,501 to 1000 Employees,2014,Company - Public,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
1,Data Scientist,$145K-$163K (Glassdoor est.),"Publishers Clearing House (PCH), located in Je...",3.3,Publishers Clearing House LLC\n3.3,"Jericho, NY",-1,501 to 1000 Employees,1953,Company - Private,Advertising & Marketing,Business Services,$500 million to $1 billion (USD),-1
2,Data Scientist – TS/SCI eligibility required,$145K-$163K (Glassdoor est.),Full-Time / Part-Time: *\nFull-Time\nData Scie...,-1,"Gnosis Solutions, Incorporated","Charlottesville, VA",-1,-1,-1,-1,-1,-1,-1,-1
3,Machine Learning Engineer,$145K-$163K (Glassdoor est.),Job Description\n\nSymetra has a great opportu...,3.2,Symetra Financial Corporation\n3.2,"Bellevue, WA",-1,1001 to 5000 Employees,1957,Company - Private,Insurance Carriers,Insurance,$2 to $5 billion (USD),-1
4,Computer Scientist,$145K-$163K (Glassdoor est.),Supporting the Most Exciting and Meaningful Mi...,2.9,"Pacific Architects & Engineers, Inc.\n2.9","Patuxent River, MD",-1,10000+ Employees,1955,Company - Public,Aerospace & Defense,Aerospace & Defense,$2 to $5 billion (USD),-1
5,"Data Scientist - Statistics, Early Career",$145K-$163K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310918\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Seattle, WA",-1,1001 to 5000 Employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),-1
6,Food Scientist,$145K-$163K (Glassdoor est.),"SUMMARY:\nThis position, under general supervi...",3.6,SupHerb Farms\n3.6,"Turlock, CA",-1,51 to 200 Employees,-1,Subsidiary or Business Segment,Food Production,Agriculture & Forestry,$10 to $25 million (USD),-1
7,Data Scientist,$145K-$163K (Glassdoor est.),Join our team dedicated to developing and exec...,4.5,Novetta\n4.5,"Annapolis Junction, MD",-1,501 to 1000 Employees,2012,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD),-1
8,"Scientist, Assay Development",$145K-$163K (Glassdoor est.),POSITION SUMMARY:\nThis position is responsibl...,3.6,Idt Biologika Gmbh\n3.6,"Rockville, MD",-1,1001 to 5000 Employees,2011,Unknown,Health Care Products Manufacturing,Manufacturing,$100 to $500 million (USD),-1
9,Machine Learning Engineer,$145K-$163K (Glassdoor est.),POSITION PURPOSE:\nThis position is responsibl...,3.9,America First Credit Union\n3.9,"Ogden, UT",-1,1001 to 5000 Employees,1939,Nonprofit Organization,Investment Banking & Asset Management,Finance,$100 to $500 million (USD),-1


In [4]:
df.to_csv("glassdoor_jobs.csv")